In [1]:
import ctypes

In [7]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [3]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f"Person({self.name})"
    
    def __del__(self):
        print(f"__del__ called for {self}")

In [4]:
p = Person("John")

In [5]:
id_p = id(p)

In [8]:
ref_count(id_p)

1

In [9]:
p = None

__del__ called for Person(John)


In [10]:
p = Person("John")
del p

__del__ called for Person(John)


In [11]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f"Person({self.name})"
    
    def __del__(self):
        print(f"__del__ called for {self}")
        
    def gen_ex(self):
        raise ValueError("Something went wrong")

In [12]:
p = Person("Eric")
p_id = id(p)
ref_count(p_id)

1

In [14]:
try:
    p.gen_ex()
except ValueError as ex:
    error = ex
    print(ex)

Something went wrong


In [15]:
error

ValueError('Something went wrong')

In [16]:
ref_count(p_id)

2

In [17]:
type(error)

ValueError

In [18]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'add_note',
 'args',
 'with_traceback']

In [19]:
dir(error.__traceback__)

['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next']

In [20]:
dir(error.__traceback__.tb_frame)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace',
 'f_trace_lines',
 'f_trace_opcodes']

In [21]:
type(error.__traceback__.tb_frame.f_locals)

dict

In [26]:
for key, value in error.__traceback__.tb_frame.f_locals.copy().items():
    if isinstance(value, Person):
        print(key, value, id(value))

p Person(Eric) 139652940150352


In [27]:
ref_count(p_id)

2

In [28]:
del p

In [29]:
del error

In [30]:
ref_count(p_id)

1

In [31]:
class Person:
    def __del__(self):
        raise ValueError("Something went wrong")

In [32]:
p = Person()

In [33]:
del p
print("All good, no exceptions")

Exception ignored in: <function Person.__del__ at 0x7f037bd86ac0>
Traceback (most recent call last):
  File "/tmp/ipykernel_8547/3337016714.py", line 3, in __del__
ValueError: Something went wrong


All good, no exceptions


In [34]:
import sys

In [35]:
sys.stderr, sys.stdout

(<ipykernel.iostream.OutStream at 0x7f037be80460>,
 <ipykernel.iostream.OutStream at 0x7f037be80550>)

In [39]:
class ErrToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stderr = sys.stderr
        
    def __enter__(self):
        self._file = open(self._fname, "w")
        sys.stderr = self._file
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stderr = self._current_stderr
        if self._file:
            self._file.close()
        return False

In [41]:
p = Person()

Exception ignored in: <function Person.__del__ at 0x7f037bd86ac0>
Traceback (most recent call last):
  File "/tmp/ipykernel_8547/3337016714.py", line 3, in __del__
ValueError: Something went wrong


In [42]:
with ErrToFile("err.txt"):
    del p
    print(100)
print(200)
print(300)

100
200
300


In [43]:
with open("err.txt") as f:
    print(f.readlines())

['Exception ignored in: <function Person.__del__ at 0x7f037bd86ac0>\n', 'Traceback (most recent call last):\n', '  File "/tmp/ipykernel_8547/3337016714.py", line 3, in __del__\n', 'ValueError: Something went wrong\n']
